In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class Alexnet(nn.Module):
    def __init__(self, num_class=1000):
        super(Alexnet, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=96, kernel_size = (11,11), stride = 4),
                          nn.ReLU(), #F.relu ----> error?
                          nn.MaxPool2d(kernel_size = (3,3), stride = 2),
                          nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k = 2))
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = (5,5), stride = 1, padding = 2, groups = 1),
                          nn.ReLU(),
                          nn.MaxPool2d(kernel_size = (3,3), stride = 2),
                          nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k = 2))
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = (3,3), stride = 1, padding = 1, groups = 1),
                                    nn.ReLU())
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = (3,3), stride = 1, padding = 1, groups = 1),
                                    nn.ReLU())
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = (3,3), stride = 1, padding = 1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size = (3,3), stride = 2)
                                   )
        self.flatten = nn.Flatten(start_dim = 1)
        self.fc2 = nn.Sequential(
                nn.Linear(4096, 4096),
                nn.ReLU())
        self.classify = nn.Sequential(
                nn.Linear(4096, num_class),
                nn.Softmax(dim=1))
        
    def fc1(self,x):
        model = nn.Sequential(
            nn.Linear(x.shape[-1], 4096),
            nn.ReLU()).to(device)
        return model(x)
        
        
                        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.classify(out)
        return out

In [4]:
model = Alexnet().to(device)

In [5]:
len(list(model.parameters()))

14

In [6]:
import torchsummary

In [7]:
torchsummary.summary(model,(3,227,227))  #input =  (batch, c, h, w)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
 LocalResponseNorm-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
         MaxPool2d-7          [-1, 256, 13, 13]               0
 LocalResponseNorm-8          [-1, 256, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,